# on January 31

In [25]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pickle
import os
import json
import numpy as np
cred = credentials.Certificate("../codeinterface-85b5e-firebase-adminsdk-11q7e-837ba92a03.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [34]:
# pilot responses
import pickle
import numpy as np
with open('/home/hussein/llm_interfaces/llm_interfaces/task_upload/saved_data/old_responses2024-01-26T19:40:48.pkl', 'rb') as f:
    responses = pickle.load(f)
emails_pilot = []
names_pilot = []
tests_pilot = ['test', 'dad', 'Wei', 'hussein2', 'Valerie']
for r in responses:
    to_skip = False
    if responses[r]['completed_task'] == 1:
        for t in tests_pilot:
            if t in responses[r]['name']:
                to_skip = True
                break        
        if not to_skip:
            emails_pilot.append(responses[r]['email'])
            names_pilot.append(responses[r]['name'])
print(len(emails_pilot))

7


In [23]:
import dateutil.parser as dparser


def process_log(study_data, type = "none"):
    frustration = int(study_data["frustration"])
    performance = int(study_data["performance"])
    temporal_demand = int(study_data["temporalDemand"])
    physical_demand = int(study_data["physicalDemand"])
    effort = int(study_data["effort"])
    mental_demand = int(study_data["mentalDemand"])
    tlx_score = get_tlx_score(frustration, performance, temporal_demand, physical_demand, effort, mental_demand)
    #print("TLX Score: ", tlx_score)


    completed_time = dparser.parse(study_data["completed_task_time"], fuzzy=True)
    date_performed = dparser.parse(study_data["date_performed"], fuzzy=True)
    study_completion = get_completion_time(date_performed, completed_time)
    #print("Completion Time: ", study_completion)


    tasks_completed = get_tasks_completed(study_data["telemetry_data"])
    #print("Tasks Completed: ", tasks_completed)

    tasks_attempted = get_tasks_attempted(study_data["telemetry_data"])
    #print("Tasks Attempted: ", tasks_attempted)

    time_to_completion, avg_time_to_completion = get_time_to_completion(study_data["telemetry_data"])
    #print("Time To Completion: ", time_to_completion)
    #print("Average Time To Completion: ", avg_time_to_completion)

    tasks_skipped = get_tasks_skipped(study_data["telemetry_data"])
    #print("Tasks Skipped: ", tasks_skipped)

    coding_time = get_coding_time(study_data["telemetry_data"])
    #print("Coding Time: ", coding_time)
    additional_metrics = {}
    if type == "autocomplete":
        accept_rate = get_suggestion_acceptance_rate(study_data["telemetry_data"])
        additional_metrics = {
            "accept_rate": accept_rate,
        }
    
    if type == "chat":
        # count 'assistant_response', 'copy_code' event types in telemetry_data
        assistant_response_count = len([event for event in study_data["telemetry_data"] if event["event_type"] == "assistant_response"])
        copy_code_count = len([event for event in study_data["telemetry_data"] if event["event_type"] == "copy_code"])
        paste_into_editor_count = len([event for event in study_data["telemetry_data"] if event["event_type"] == "paste_into_editor"])
        additional_metrics = {
            "assistant_response_count": assistant_response_count,
            "copy_code_count": copy_code_count,
            "paste_into_editor_count": paste_into_editor_count,
        }
    	
    if type == "chat" or type == "autocomplete":
        # aihelpful 
        aihelpful = int(study_data["aihelpful"])
        additional_metrics["aihelpful"] = aihelpful

    dict_metrics  = {

        "tlx_score": tlx_score,
        "study_completion": study_completion,
        "tasks_completed": tasks_completed,
        "tasks_attempted": tasks_attempted,
        "time_to_completion": time_to_completion,
        "avg_time_to_completion": avg_time_to_completion,
        "tasks_skipped": tasks_skipped,
        "coding_time": coding_time,
        **additional_metrics
    }
    return dict_metrics

def get_tlx_score(frustration, performance, temporal_demand, physical_demand, effort, mental_demand):
    return (frustration + performance + temporal_demand + physical_demand + effort + mental_demand) * 5


def convert_tool_usage_to_str(tool_usage):
    if tool_usage == "1":
        return "Strongly Disagree"
    elif tool_usage == "2":
        return "Disagree"
    elif tool_usage == "3":
        return "Neutral"
    elif tool_usage == "4":
        return "Agree"
    elif tool_usage == "5":
        return "Strongly Agree"
    else:
        raise ValueError("Invalid tool usage")


def get_completion_time(start_time, end_time):
    return end_time - start_time


def get_suggestion_acceptance_rate(telemetry_data):
    num_accept = len([event for event in telemetry_data if event["event_type"] == "accept"])
    # only count suggestion_shown when suggestion is not ""
    num_suggestion_shown = len([event for event in telemetry_data if event["event_type"] == "suggestion_shown" and event["suggestion"] != ""])
    # if num_suggestion_shown == 0:
    #     print("No suggestions shown!!")
    #     return np.nan
    
    return num_accept / num_suggestion_shown


def get_tasks_completed(telemetry_data):
    return len(
        [event for event in telemetry_data if event["event_type"] == "submit_code" and event["completed_task"] == 1 and event["task_index"] != -1]
    )


def get_tasks_attempted(telemetry_data):
    return len([event for event in telemetry_data if event["event_type"] == "load_task"])


def get_time_to_completion(telemetry_data):
    starts = [event for event in telemetry_data if event["event_type"] == "load_task"]
    ends = [
        event#["timestamp"]
        for event in telemetry_data
        if event["event_type"] == "submit_code" and event["completed_task"] == 1
    ]
    times_tasks_solved = []
    task_indices_seen = set()
    for start in starts:
        if start["task_index"] == -1 or start["task_index"] in task_indices_seen:
            continue
        task_indices_seen.add(start["task_index"])
        
        for end in ends:
            if end["task_index"] == start["task_index"]:
                # check if tim is more than 10mins
                if (end["timestamp"] - start["timestamp"]) / 1000 < 600:
                    times_tasks_solved.append((end["timestamp"] - start["timestamp"]) / 1000)
                    break


    if len(times_tasks_solved) == 0:
        return 0, np.nan
    return times_tasks_solved, np.mean(times_tasks_solved)


def get_coding_time(telemetry_data):
    # Get first load task
    start = [event["timestamp"] for event in telemetry_data if event["event_type"] == "load_task"][0]

    # Get last telemetry event
    end = telemetry_data[-1]["timestamp"]

    return (end - start) / 1000


def get_tasks_skipped(telemetry_data):
    return len([event for event in telemetry_data if event["event_type"] == "skip_task"])


def get_tasks_skipped(telemetry_data):
    return len([event for event in telemetry_data if event["event_type"] == "skip_task"])


def get_time_verifying_suggestion(telemetry_data):
    # Get suggestions
    suggestions_shown = [event for event in telemetry_data if event["event_type"] == "suggestion_shown"]

    suggestions_reviewed = [
        event for event in telemetry_data if event["event_type"] == "reject" or event["event_type"] == "accept"
    ]

    # Create a hashmap for suggestion reviews.
    reviewed_hashmap = {}
    for event in suggestions_reviewed:
        reviewed_hashmap[event["suggestion_id"]] = event["timestamp"]

    # Create a hashmap for times to completion
    time_spent_verifying = {}
    for event in suggestions_shown:
        if event["suggestion_id"] in reviewed_hashmap:
            time_spent_verifying[event["suggestion_id"]] = (
                reviewed_hashmap[event["suggestion_id"]] - event["timestamp"]
            ) / 1000
        else:
            print("No review found for suggestion: ", event["suggestion_id"])

    return time_spent_verifying


In [30]:
docs = db.collection('responses').get()
# save docs in a pickle file called old_tasks.pkl
responses = {}
all_responses = {}
done_count = 0
emails_first = []
names_first = []
for doc in docs:
    doc_dict = doc.to_dict()
    doc_dict['id'] = doc.id 
    resp = doc_dict
    resp_id = resp['id']
    #print(resp_id)
    if 'entered_exit_survey' not in resp or 'finalcomments' not in resp:
        continue
    log_metrics = process_log(resp)

    if 'test' in resp['name'] or 'test' in resp['email']:
        continue
    if 'Nathan Gandawa' in resp['name'] or 'nathangandawa' in resp['email']:
        continue
    if 'leek' in resp['name'] or 'leek' in resp['email']:
        continue
    if log_metrics['tasks_completed'] < 2:
        continue
    print(f'name {resp["name"]} email {resp["email"]} id {resp_id} completed {log_metrics["tasks_completed"]} tasks')
    done_count += 1
    emails_first.append(resp['email'])
    names_first.append(resp['name'])

print(done_count)

name Ming Chong Lim email mingchol@andrew.cmu.edu id autocomplete_gpt35_0_4-4767663 completed 7 tasks
name San Shin email samshin0714@gmail.com id autocomplete_gpt35_0_9-263157 completed 2 tasks
name Shrikara Varna email svarna@andrew.cmu.edu id autocomplete_gpt35_1_1-5457944 completed 3 tasks
name Nirajan Koirala email nkoirala@nd.edu id autocomplete_gpt35_1_10-7542461 completed 3 tasks
name Eric Schneider email franz.eric.schneider@gmail.com id autocomplete_gpt35_1_2-7342708 completed 6 tasks
name Jennifer Guo email gwyjennifer@gmail.com id autocomplete_gpt35_1_3-4590470 completed 5 tasks
name Che-Ping Tsai email chepingt@andrew.cmu.edu id autocomplete_gpt35_1_7-531112 completed 5 tasks
name Alfredo Gomez email alfredogomez58@gmail.com id autocomplete_gpt35_2_1-3570641 completed 2 tasks
name Glenn Chentianye Xu email chentiax@andrew.cmu.edu id autocomplete_gpt35_2_7-2692440 completed 3 tasks
name Edward Jin email xdanieh1@gmail.com id autocomplete_gpt35_4_0-1918441 completed 7 tasks


In [31]:
# combine emails_first and emails_pilot 
emails = emails_first + emails_pilot
names = names_first + names_pilot


In [32]:
# print comma joined emails and names
print(','.join(emails))

mingchol@andrew.cmu.edu,samshin0714@gmail.com,svarna@andrew.cmu.edu,nkoirala@nd.edu,franz.eric.schneider@gmail.com,gwyjennifer@gmail.com,chepingt@andrew.cmu.edu,alfredogomez58@gmail.com,chentiax@andrew.cmu.edu,xdanieh1@gmail.com,ashibusa@andrew.cmu.edu,maliknamya.20@dartmouth.edu,aryas@andrew.cmu.edu,ziweigu@g.harvard.edu,upriyam@andrew.cmu.edu,yixuanz2@andrew.cmu.edu,lamanda@mit.edu,akashkan@cs.cmu.edu,yimingz3@cs.cmu.edu,razikgrewal@gmail.com,dereklim@mit.edu,samtenka@umich.edu,tslilyai@mit.edu,etracey@nd.edu,rguo48@gatech.edu,tutrinh@berkeley.edu,marietheresahernandez@gmail.com,therealvikramjeet@gmail.com,mislam6@ur.rochester.edu,vdandeka@andrew.cmu.edu,jjzhao2@andrew.cmu.edu,shrey2809@gmail.com,ptinsley@nd.edu,cyt046@gmail.com,llee23@nd.edu,manhbaon@cs.cmu.edu,elainewang2001@gmail.com,julialujiawang@gmail.com,dutsinm@gmail.com,yooni.jy.choi@gmail.com,spkrishnaofficial@gmail.com,anasery@uw.edu,markakis@mit.edu,lylakirati@gmail.com,sfish@g.harvard.edu,mckay.sjm98@gmail.com,pradyumna_

In [33]:
print(','.join(names))

Ming Chong Lim,San Shin,Shrikara Varna,Nirajan Koirala,Eric Schneider,Jennifer Guo,Che-Ping Tsai,Alfredo Gomez,Glenn Chentianye Xu,Edward Jin,Ankit Shibusam,Dhruv Malik,Arya Shah,Ziwei Gu,Utkarsh Priyam,Alan Zhu,Amanda Liu,Akash Kannan,Yiming Zhang,RAZIK SINGH GREWAL,Derek Lim,Samuel Tenka,Lily Tsai,Eamon Tracey,Ruohao Guo,Tu Trinh,marie hernandez,Vikramjeet Das,Md Saiful Islam,Ved Dandekar,James Zhao,Shrey Gupta,Patrick Tinsley,Yishen Chen,Lauren Lee,Manh-Bao Nguyen,Elaine Wang,Julia Wang,Dennis Mark,Yooni Choi,Satyapriya Krishna,Anshul Nasery,Markos Markakis,Lanlana Kiratiwudhikul,Sara Fish,Stephen McKay,Sai Pradyumna Chermala,Harshal Chamdal,Kyle Johnsen,Vincent Tombari,Marc Descoteaux,Michael Yang,Christy Huang,Yilin Zhou,Tianmai Zhang,Ajay Mittur,Simon Mendelsohn,Aryan Singhal,Pearl Levell


In [35]:
print(len(emails))

59
